### Key features
- SQL syntax highlighting for improved readability
- Direct integration with Spark SQL for optimized execution
- Seamless interoperability between Python and SQL

**By combining SQL's declarative power with Python's flexibility, this approach offers:**
- Enhanced developer experience through immediate visual validation
- Improved governance and auditability of data transformations
- Efficient debugging and knowledge transfer within data teams
- Maintained performance through native Spark SQL execution

In [0]:
pip install SQLGlassBoxFunction

Python interpreter will be restarted.
  Using cached SQLGlassBoxFunction-0.1.4-py3-none-any.whl (5.3 kB)
Python interpreter will be restarted.


In [0]:
from SQLGlassBoxFunction import sql

In [0]:
import os
os.environ['SQL_GBF_WRITE_TO_LOG_FILE'] = 'True'
os.environ['SQL_GBF_DRY_RUN'] = 'False'
os.environ['SQL_GBF_QUERY_LOG_FILE'] = 'sql_log_demo.txt'

In [0]:
%run ./SourceData

In [0]:
%run ./NewApproach-Commons

##Example function call

In [0]:
# Assuming the initial data is loaded into a temporary view called "input_vw"
input_vw = "vw_SourceData"  # Replace with the actual input view name

# Apply all validations in sequence
vw_with_order_id = validate_order_id(input_vw)


        CREATE OR REPLACE TEMP VIEW vw_validate_order_id AS
        SELECT  * EXCEPT (validity),
        CASE
            WHEN order_id <= 0 THEN 'Invalid Order ID'
            ELSE validity 
        END AS validity
        FROM vw_SourceData;
    
Log file path: sql_log_demo.txt
SQL Log File location: /databricks/driver/sql_log_demo.txt


### Benefit 1 : Transparency helps in quick knowlwdge transfer

In [0]:
vw_with_quantity = validate_quantity(vw_with_order_id)


        CREATE OR REPLACE TEMP VIEW vw_validate_quantity AS
        SELECT  * EXCEPT (validity),
        CASE
            WHEN quantity <= 0 THEN 'Invalid Quantity'
            ELSE validity 
        END AS validity
        FROM vw_validate_order_id;
    
Log file path: sql_log_demo.txt
SQL Log File location: /databricks/driver/sql_log_demo.txt


### Benefit 2 : Perform easy logging of transformation logics for future audit or debugging issues

In [0]:
vw_with_phone_number = validate_phone_number(vw_with_quantity)


        CREATE OR REPLACE TEMP VIEW vw_validate_phone_number AS
        SELECT  * EXCEPT (validity),
        CASE
            WHEN LENGTH(phone_number) != 10 THEN 'Invalid Phone Number'
            ELSE validity 
        END AS validity
        FROM vw_validate_quantity;
    
Log file path: sql_log_demo.txt
SQL Log File location: /databricks/driver/sql_log_demo.txt


### Benefit 3 : Easy to analyze and perform debugging and enhancements

In [0]:
vw_with_email = validate_email(vw_with_phone_number)


        CREATE OR REPLACE TEMP VIEW vw_validate_email AS
        SELECT  * EXCEPT (validity),
        CASE
            WHEN customer_email IS NULL OR customer_email = '' THEN 'Missing Email'
            ELSE validity 
        END AS validity
        FROM vw_validate_phone_number;
    
Log file path: sql_log_demo.txt
SQL Log File location: /databricks/driver/sql_log_demo.txt


### Benefit 4 : Syntax higlighting SQL for better readability

In [0]:
vw_with_order_date = validate_order_date(vw_with_email)


        CREATE OR REPLACE TEMP VIEW vw_validate_order_date AS
        SELECT  * EXCEPT (validity),
        CASE
            WHEN order_date > CURRENT_DATE THEN 'Order Date is in the Future'
            ELSE validity 
        END AS validity
        FROM vw_validate_email;
    
Log file path: sql_log_demo.txt
SQL Log File location: /databricks/driver/sql_log_demo.txt


### Benefit 5: Make easy decisions without too much digging

In [0]:
vw_with_discount_percentage = validate_discount_percentage(vw_with_order_date)

# The final view will have the validity column populated
# You can query the final view like:
display(spark.sql(f"SELECT validity,* FROM {vw_with_discount_percentage}"))


        CREATE OR REPLACE TEMP VIEW vw_validate_discount_percentage AS
        SELECT  * EXCEPT (validity),
        CASE
            WHEN discount_percentage < 0 OR discount_percentage > 50 THEN 'Invalid Discount Percentage'
            ELSE validity 
        END AS validity
        FROM vw_validate_order_date;
    
Log file path: sql_log_demo.txt
SQL Log File location: /databricks/driver/sql_log_demo.txt


validity,order_id,customer_name,customer_email,phone_number,order_date,total_amount,quantity,discount_percentage,validity
valid,1,John Doe,john.doe@example.com,1234567890,2025-02-06,150.0,3,10,valid
Invalid Quantity,2,Jane Smith,jane.smith@example.com,9876543210,2025-02-06,200.0,-2,15,Invalid Quantity
Missing Email,3,Alice Johnson,,1112233445,2025-02-06,120.0,5,30,Missing Email
Invalid Discount Percentage,4,Bob Lee,bob.lee@example.com,12345abcde,2025-02-06,90.0,4,60,Invalid Discount Percentage
valid,5,Charlie Brown,charlie.brown@example.com,5551234567,2025-02-07,250.0,1,20,valid
Invalid Discount Percentage,6,David Clark,david.clark@example.com,1112233445,2025-02-06,180.0,3,-10,Invalid Discount Percentage
valid,7,Emma White,emma.white@example.com,3334445555,2025-02-06,100.0,2,25,valid
